In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
import pandas as pd


In [2]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%%time
zip_path = '/content/drive/My Drive/Colab Notebooks/allimgs_face2.zip'
!cp "{zip_path}" .
!unzip -q allimgs_face2.zip 
!rm allimgs_face2.zip

CPU times: user 56.9 ms, sys: 18.3 ms, total: 75.2 ms
Wall time: 14.5 s


In [4]:
# # Get train and test data for classification
train_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_on_cand.csv')
new_paths = []
for path, winner in zip(train_df[train_df.columns[2]].tolist(), train_df[train_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths.append([new_path, winner])
train_df = pd.DataFrame(new_paths) 
train_df[train_df.columns[1]]= train_df[train_df.columns[1]].astype(float)
# train_df = train_df.sample(frac=1).reset_index(drop=True)


test_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_on_cand.csv')
new_paths_test = []
for path, winner in zip(test_df[test_df.columns[2]].tolist(), test_df[test_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths_test.append([new_path, winner])

test_df = pd.DataFrame(new_paths_test)    
test_df[test_df.columns[1]]= test_df[test_df.columns[1]].astype(float)   

val_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/val_on_cand.csv')
new_paths_val = []
for path, winner in zip(val_df[val_df.columns[2]].tolist(), val_df[val_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths_val.append([new_path, winner])


val_df = pd.DataFrame(new_paths_val)    
val_df[val_df.columns[1]]= val_df[val_df.columns[1]].astype(float)   
# val_df

In [5]:
# Create a dataloader class
class DatasetFaces_Rank(Dataset):
    
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        img_name = self.data[self.data.columns[0]].iloc[index]
        # img_name = img_name.split('/')

   

        image = Image.open("/content/"+img_name).convert('RGB')
        # label = self.data[self.data.columns[1]].iloc[index]
        if self.transform is not None:
            image = self.transform(image)
        else:
            print("TRANSFORM FAILED")
        return image
  
# Create a dataloader class
class DatasetFaces_classify(Dataset):
    
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        img_name = self.data[self.data.columns[0]].iloc[index]
        # img_name = img_name.split('/')

   

        image = Image.open("/content/"+img_name).convert('RGB')
        label = self.data[self.data.columns[1]].iloc[index]
        if self.transform is not None:
            image = self.transform(image)
        else:
            print("TRANSFORM FAILED")
        return image,label

In [6]:
# Create a transform for data prep
transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])


# Classification
train_dataset = DatasetFaces_classify(train_df, transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1,shuffle=False, num_workers=1)

test_dataset = DatasetFaces_classify(test_df, transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False, num_workers=1)


val_dataset = DatasetFaces_classify(val_df, transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,shuffle=False, num_workers=1)


In [7]:
# CHANGE THESE TO FINETUNED WEIGHTS
# pretrained_weights = torch.load('/content/drive/My Drive/Colab Notebooks/resnet50_scratch_dims_2048.pth')
# model = Resnet50_scratch()
# model.load_state_dict(pretrained_weights)

model = models.resnet50(pretrained = False)
model.fc = nn.Sequential(
    nn.Dropout(0.7),
    nn.Linear(2048, 1024),
    nn.Dropout(0.8),
    nn.Linear(1024, 1),
    nn.Sigmoid()
)

weights = torch.load('/content/drive/My Drive/Colab Notebooks/model weights/vggface_resnet_3conv_best.pl')
model.load_state_dict(weights)
print(model)

feature_extractor_nine = torch.nn.Sequential(*(list(model.children())[:-2]))
feature_extractor_eight = torch.nn.Sequential(*(list(model.children())[:-3]))
feature_extractor_seven = torch.nn.Sequential(*(list(model.children())[:-4]))
if torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("done")
# print(feature_extractor_nine)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# train_features = []
# test_features = []
val_features = []

# with torch.no_grad():
#         i = 0
#         for data in train_loader:
#             device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#             inputs, label = data
#             inputs = inputs.to(device)

#             l7 = feature_extractor_seven(inputs)
#             l8 = feature_extractor_eight(inputs)
#             l9 = feature_extractor_nine(inputs)
#             m7 = nn.MaxPool2d(32, stride=1)
#             m8 = nn.MaxPool2d(16, stride=1)
#             m9 = nn.MaxPool2d(8, stride=1)
#             l7 = m7(l7)
#             l8 = m8(l8)
#             l9 = m9(l9)
#             train_features.append([l7.cpu().numpy(),l8.cpu().numpy(),l9.cpu().numpy(), label])
       
# with torch.no_grad():
#         i = 0
#         for data in test_loader:
#             device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#             inputs, label = data
#             inputs = inputs.to(device)

#             l7 = feature_extractor_seven(inputs)
#             l8 = feature_extractor_eight(inputs)
#             l9 = feature_extractor_nine(inputs)
#             m7 = nn.MaxPool2d(32, stride=1)
#             m8 = nn.MaxPool2d(16, stride=1)
#             m9 = nn.MaxPool2d(8, stride=1)
#             l7 = m7(l7)
#             l8 = m8(l8)
#             l9 = m9(l9)
#             test_features.append([l7.cpu().numpy(),l8.cpu().numpy(),l9.cpu().numpy(), label])
with torch.no_grad():
        i = 0
        for data in val_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs, label = data
            inputs = inputs.to(device)

            l7 = feature_extractor_seven(inputs)
            l8 = feature_extractor_eight(inputs)
            l9 = feature_extractor_nine(inputs)
            m7 = nn.MaxPool2d(32, stride=1)
            m8 = nn.MaxPool2d(16, stride=1)
            m9 = nn.MaxPool2d(8, stride=1)
            l7 = m7(l7)
            l8 = m8(l8)
            l9 = m9(l9)
            val_features.append([l7.cpu().numpy(),l8.cpu().numpy(),l9.cpu().numpy(), label])
            


In [11]:
# train_features = pd.DataFrame(train_features)
train_features.to_pickle('/content/drive/My Drive/Colab Notebooks/resnet_vggface_train.pl')
# test_features = pd.DataFrame(test_features)
test_features.to_pickle('/content/drive/My Drive/Colab Notebooks/resnet_vggface_test.pl')
# val_features = pd.DataFrame(val_features)
val_features.to_pickle('/content/drive/My Drive/Colab Notebooks/resnet_vggface_val.pl')

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
